# Pydantic Demo

### Python Type Hinting

In [44]:
# Static type hinting (not enforced)

def square(x: int) -> float:
    return x**2


square(3.5)

12.25

### BaseModel

In [49]:
from typing import List, Literal, Optional
from pydantic import BaseModel, SecretStr, HttpUrl, Field


class Column(BaseModel):
    name: str
    dtype: Literal["integer", "float", "string", "boolean"]
    primary_key: bool = False
    description: Optional[str] = None


class Table(BaseModel):
    name: str
    columns: List[Column]


class Database(BaseModel):
    connection_string: HttpUrl
    user: str = Field(strip_whitespace=True)
    password: SecretStr
    tables: List[Table]

In [52]:
# Valid data
my_table = Table(
    name="my_table",
    columns=[
        Column(name="field1", dtype="string", primary_key=True),
        Column(name="field2", dtype="integer", description="My description")
    ]
)

my_db = Database(
    connection_string="https://www.rsginc.com:9999",
    user="me",
    password="CorrectHorseBatteryStaple",
    tables=[my_table],
)

In [58]:
from pprint import pprint

# Check out values.  Notice that password is not displayed
pprint(dict(my_db))

{'connection_string': HttpUrl('https://www.rsginc.com:9999', scheme='https', host='www.rsginc.com', tld='com', host_type='domain', port='9999'),
 'password': SecretStr('**********'),
 'tables': [Table(name='my_table', columns=[Column(name='field1', dtype='string', primary_key=True, description=None), Column(name='field2', dtype='integer', primary_key=False, description='My description')])],
 'user': 'me'}


In [59]:
# Print schema
my_db.schema()

{'title': 'Database',
 'type': 'object',
 'properties': {'connection_string': {'title': 'Connection String',
   'minLength': 1,
   'maxLength': 2083,
   'format': 'uri',
   'type': 'string'},
  'user': {'title': 'User', 'strip_whitespace': True, 'type': 'string'},
  'password': {'title': 'Password',
   'type': 'string',
   'writeOnly': True,
   'format': 'password'},
  'tables': {'title': 'Tables',
   'type': 'array',
   'items': {'$ref': '#/definitions/Table'}}},
 'required': ['connection_string', 'user', 'password', 'tables'],
 'definitions': {'Column': {'title': 'Column',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'dtype': {'title': 'Dtype',
     'enum': ['integer', 'float', 'string', 'boolean'],
     'type': 'string'},
    'primary_key': {'title': 'Primary Key',
     'default': False,
     'type': 'boolean'},
    'description': {'title': 'Description', 'type': 'string'}},
   'required': ['name', 'dtype']},
  'Table': {'title': 'Table',
 

In [53]:
my_db.password

SecretStr('**********')

In [54]:
my_db.password._secret_value

'CorrectHorseBatteryStaple'

### In Functions

In [64]:
# Use @validate_arguments decorator
from pydantic import validate_arguments


@validate_arguments
def my_function(crs: CRS, foo = None, bar = None):
    return crs

In [65]:
# Valid
my_function("EPSG:4326")

'EPSG:4326'

In [66]:
# Invalid
my_function("MY_STRING")

ValidationError: 1 validation error for MyFunction
crs
  invalid CRS! (type=value_error)

### Validators

### Constrained Types

#### Numerics

In [67]:
from pydantic import (
    NegativeFloat,
    NegativeInt,
    NonNegativeFloat,
    NonNegativeInt,
    NonPositiveFloat,
    NonPositiveInt,
    PositiveFloat,
    PositiveInt,
)


class MyModel(BaseModel):
    n_float: NegativeFloat  # < 0
    n_int: NegativeInt  # < 0
    nn_float: NonNegativeFloat  # >= 0
    nn_int: NonNegativeInt  # >= 0
    np_float: NonPositiveFloat  # <= 0
    np_int: NonPositiveInt  # <= 0
    p_float: PositiveFloat  # > 0
    p_int: PositiveInt  # > 0


# Valid
my_instance = MyModel(
    n_float=-1.5,
    n_int=-1,
    nn_float=0,  # will be coerced to float unless we specify strict to be True
    nn_int=0,
    np_float=0,
    np_int=0,
    p_float=3.14,
    p_int=10,
)

In [68]:
# Invalid
my_instance=MyModel(
    n_float=0,
    n_int=0,
    nn_float=-1.5,
    nn_int=-1,
    np_float=1.5,
    np_int=1,
    p_float=0,
    p_int=0,
)

ValidationError: 8 validation errors for MyModel
n_float
  ensure this value is less than 0 (type=value_error.number.not_lt; limit_value=0)
n_int
  ensure this value is less than 0 (type=value_error.number.not_lt; limit_value=0)
nn_float
  ensure this value is greater than or equal to 0 (type=value_error.number.not_ge; limit_value=0)
nn_int
  ensure this value is greater than or equal to 0 (type=value_error.number.not_ge; limit_value=0)
np_float
  ensure this value is less than or equal to 0 (type=value_error.number.not_le; limit_value=0)
np_int
  ensure this value is less than or equal to 0 (type=value_error.number.not_le; limit_value=0)
p_float
  ensure this value is greater than 0 (type=value_error.number.not_gt; limit_value=0)
p_int
  ensure this value is greater than 0 (type=value_error.number.not_gt; limit_value=0)

### File, Directory Paths

In [71]:
from pydantic import DirectoryPath, FilePath


class MyModel(BaseModel):
    my_directory: DirectoryPath  # directory must exist
    my_file: FilePath  # file must exist


# Valid
my_instance = MyModel(my_directory="secrets", my_file="Dockerfile")

In [72]:
# Invalid
my_instance = MyModel(my_directory="missing_directory", my_file="missing_file")

ValidationError: 2 validation errors for MyModel
my_directory
  file or directory at path "missing_directory" does not exist (type=value_error.path.not_exists; path=missing_directory)
my_file
  file or directory at path "missing_file" does not exist (type=value_error.path.not_exists; path=missing_file)

### Strings

In [79]:
from pydantic import FutureDate, PaymentCardNumber
from pydantic.types import PaymentCardBrand


class MyModel(BaseModel):
    first: str = Field(strip_whitespace=True, min_length=1)
    last: str = Field(strip_whitespace=True, min_length=1)
    exp: FutureDate
    cc_no: PaymentCardNumber
    postalcd: str = Field(strip_whitespace=True, regex=r"^\d{5}(?:\-\d{4})?$")

    @property
    def cc_brand(self) -> PaymentCardBrand:
        return self.cc_no.brand


# valid
my_instance = MyModel(
    first="Matt",
    last="Morris",
    exp="2099-12-31",
    cc_no="4000000000000002",
    postalcd="20500",
)

print(my_instance.cc_no.brand)
print(my_instance.cc_no.bin)
print(my_instance.cc_no.last4)
print(my_instance.cc_no.masked)

Visa
400000
0002
400000******0002


In [81]:
# Invalid
my_instance = MyModel(
    first="",
    last="",
    exp="2000-01-01",
    cc_no="1234",
    postalcd="would't you like to know",
)

ValidationError: 5 validation errors for MyModel
first
  ensure this value has at least 1 characters (type=value_error.any_str.min_length; limit_value=1)
last
  ensure this value has at least 1 characters (type=value_error.any_str.min_length; limit_value=1)
exp
  date is not in the future (type=value_error.date.not_in_the_future)
cc_no
  ensure this value has at least 12 characters (type=value_error.any_str.min_length; limit_value=12)
postalcd
  string does not match regex "^\d{5}(?:\-\d{4})?$" (type=value_error.str.regex; pattern=^\d{5}(?:\-\d{4})?$)

### Custom Types

In [83]:
from pydantic import ConstrainedFloat, Field


# Number between 0 and 1
class Proportion(ConstrainedFloat):
    ge=0
    le=1


@validate_arguments
def my_function(id_var: Proportion):
    return id_var


# Valid
my_function(0.5)

0.5

In [84]:
# Invalid
my_function(2)

ValidationError: 1 validation error for MyFunction
id_var
  ensure this value is less than or equal to 1 (type=value_error.number.not_le; limit_value=1)

In [86]:
# CRS (from rxy pipeline)
import re


CRS_REGEX = r"(EPSG|ESRI|SR-ORG|IAU2000):\d{2,8}"


class CRS(str):
    """
    String formatted as legitimate CRS with custom validation
    (see https://pydantic-docs.helpmanual.io/usage/types/#custom-data-types)

    Usage:

    ```python
    # In function
    from pydantic import validate_arguments

    @validate_arguments
    def my_func(crs: CRS, ...):
        # Do something with crs
        ...

    # In BaseModel
    from pydantic import BaseModel

    class MyModel(BaseModel):
        crs: CRS
        ...
    ```
    """

    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def __modify_schema__(cls, field_schema):
        field_schema.update(
            # valid regex
            pattern=CRS_REGEX,
            # some example CRS's
            examples=["EPSG:4326", "ESRI:102310"],
        )

    @classmethod
    def validate(cls, v):
        if not isinstance(v, str):
            raise TypeError("string required!")
        if not re.match(CRS_REGEX, v):
            raise ValueError("invalid CRS!")
        return v

    def __repr__(self):
        return f"CRS({super().__repr__()})"


class MyModel(BaseModel):
    crs: CRS

In [87]:
# Valid
my_instance = MyModel(crs="EPSG:4326")

In [88]:
# Valid
my_instance = MyModel(crs="MY_STRING")

ValidationError: 1 validation error for MyModel
crs
  invalid CRS! (type=value_error)

### BaseSettings

In [89]:
# We'll read some values from local files

from pydantic import BaseSettings


# Revisit Database, but now subclass `BaseSettings` instead of `BaseModel`
class Database(BaseSettings):
    connection_string: HttpUrl
    user: str
    password: SecretStr
    tables: List[Table]

    class Config:
        # Give option to specify values in a secrets directory
        secrets_dir = "secrets"  # directory with secrets values
        env_file = ".env"  # path to .env file
        env_file_encoding = "utf-8"


my_db = Database(connection_string="https://www.rsginc.com", tables=[my_table])

In [90]:
my_db.password._secret_value

'mypass'

### Other

In [92]:
# View json schema
Database.schema_json()

'{"title": "Database", "description": "Base class for settings, allowing values to be overridden by environment variables.\\n\\nThis is useful in production for secrets you do not wish to save in code, it plays nicely with docker(-compose),\\nHeroku and any 12 factor app design.", "type": "object", "properties": {"connection_string": {"title": "Connection String", "env_names": ["connection_string"], "minLength": 1, "maxLength": 2083, "format": "uri", "type": "string"}, "user": {"title": "User", "env_names": ["user"], "type": "string"}, "password": {"title": "Password", "env_names": ["password"], "type": "string", "writeOnly": true, "format": "password"}, "tables": {"title": "Tables", "env_names": ["tables"], "type": "array", "items": {"$ref": "#/definitions/Table"}}}, "required": ["connection_string", "user", "password", "tables"], "additionalProperties": false, "definitions": {"Column": {"title": "Column", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "

In [100]:
# Generate instance from config file
my_db = Database.parse_file("database.json")

In [103]:
# Generate instance from Python dictionary
import yaml
from pathlib import Path

# Get data from .yaml file
with Path("database.yaml").open("r") as file:
    data = yaml.safe_load(file)

my_db = Database.parse_obj(data)